In [124]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
import os
import pickle
%matplotlib inline
from datetime import datetime
from dateutil.parser import parse

In [125]:
shift_month_d  = -3

In [126]:
with open("apartment_sale.pkl", 'rb') as picklefile: 
    zillow_apartment_df = pickle.load(picklefile)
zillow_apartment_df.head()


,RegionName,State,Metro,CountyName,SizeRank,BedSize,Year,Month,Amount
18,Seattle,WA,Seattle,King,19,Studio,2011,03,1097.5
18,Seattle,WA,Seattle,King,19,Studio,2011,04,1095.0
18,Seattle,WA,Seattle,King,19,Studio,2011,05,1025.0
18,Seattle,WA,Seattle,King,19,Studio,2011,06,995.0
18,Seattle,WA,Seattle,King,19,Studio,2011,07,1000.0


In [127]:
studio_df = zillow_apartment_df.loc[zillow_apartment_df['BedSize'] == 'Studio']
studio_df['Year'].value_counts()

2015    12
2014    12
2013    12
2012    12
2016    12
2017    12
2011    10
2018     2
Name: Year, dtype: int64

In [128]:
zillow_apartment_df.shape

(302, 9)

In [129]:
with open("google_trend.pkl", 'rb') as picklefile: 
    google_trend_df = pickle.load(picklefile)

google_trend_df.head()
k = google_trend_df.groupby(['Year', 'Month']).count()


In [130]:
google_trend_df ['Date'] = pd.to_datetime((google_trend_df['Year'] + '-' + google_trend_df['Month']+ '-01') ,format='%Y-%m-%d')
google_trend_df.shape

(516, 5)

In [131]:

date_shift_six_month_df = google_trend_df.Date+   pd.offsets.DateOffset(months=shift_month_d)
google_trend_df ['Shift_Year'] = date_shift_six_month_df.dt.year
google_trend_df ['Shift_Month'] = date_shift_six_month_df.dt.month.map("{:02}".format)
google_trend_df.shape

(516, 7)

In [132]:
google_trend_df_SeattleRent = google_trend_df.loc[google_trend_df['Term'] == 'SeattleRent']
google_trend_df_SeattleRent.rename(columns={"Amount": "Search_Count_SeattleRent"}, inplace=True)
google_trend_df_SeattleRent = google_trend_df_SeattleRent.drop(['Term', 'Year','Month', 'Date'], 1)

google_trend_df_SeattleHousing = google_trend_df.loc[google_trend_df['Term'] == 'SeattleHousing']
google_trend_df_SeattleHousing.rename(columns={"Amount": "Search_Count_SeattleHousing"}, inplace=True)
google_trend_df_SeattleHousing = google_trend_df_SeattleHousing.drop(['Term', 'Year','Month', 'Date'], 1)

google_trend_df_seattleApartment = google_trend_df.loc[google_trend_df['Term'] == 'seattleApartment']
google_trend_df_seattleApartment.rename(columns={"Amount": "Search_Count_seattleApartment"}, inplace=True)
google_trend_df_seattleApartment = google_trend_df_seattleApartment.drop(['Term', 'Year','Month', 'Date'], 1)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [133]:
google_trend_df_SeattleRent.tail()

,Search_Count_SeattleRent,Shift_Year,Shift_Month
167,31,2017,09
168,39,2017,10
169,38,2017,11
170,38,2017,12
171,40,2018,01


In [134]:
with open("zillow_housing.pkl", 'rb') as picklefile: 
    zillow_housing_df = pickle.load(picklefile)
zillow_housing_df.head()

,RegionID,RegionName,StateName,SizeRank,Year,Month,Amount
17,16037.0,Seattle,Washington,18,2008,03,421300.0
17,16037.0,Seattle,Washington,18,2008,04,424000.0
17,16037.0,Seattle,Washington,18,2008,05,426000.0
17,16037.0,Seattle,Washington,18,2008,06,422900.0
17,16037.0,Seattle,Washington,18,2008,07,414400.0


In [135]:
zillow_housing_df ['Date'] = pd.to_datetime((zillow_housing_df['Year'] + '-' + zillow_housing_df['Month']+ '-01') ,format='%Y-%m-%d')
zillow_housing_df_shift_six_month_df = zillow_housing_df.Date+   pd.offsets.DateOffset(months=shift_month_d)
zillow_housing_df ['Shift_Year'] = zillow_housing_df_shift_six_month_df.dt.year.astype(str)
zillow_housing_df ['Shift_Month'] = zillow_housing_df_shift_six_month_df.dt.month.map("{:02}".format)
zillow_housing_df = zillow_housing_df.drop(['RegionID','RegionName','StateName','SizeRank','Year','Month', 'Date'], 1)


zillow_housing_df.head()

,Amount,Shift_Year,Shift_Month
17,421300.0,2007,12
17,424000.0,2008,01
17,426000.0,2008,02
17,422900.0,2008,03
17,414400.0,2008,04


In [136]:
zillow_apartment_df['Shift_Year'] = zillow_apartment_df['Year'].astype(str)
zillow_apartment_df['Shift_Month'] = zillow_apartment_df['Month'].astype(str)

In [137]:
merge_housing = pd.merge(zillow_apartment_df, zillow_housing_df, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing = merge_housing.rename(columns={'Amount_y':'Zillow_Housing_price'})
merge_housing = merge_housing.rename(columns={'Amount_x':'Apartment_rent'})


merge_housing.shape

(302, 12)

In [138]:
google_trend_df_SeattleRent['Shift_Year'] = google_trend_df_SeattleRent['Shift_Year'].astype(str)
google_trend_df_SeattleRent['Shift_Month'] = google_trend_df_SeattleRent['Shift_Month'].astype(str)
k = google_trend_df_SeattleRent.groupby(['Shift_Year', 'Shift_Month']).count()
k.head(1)


,,Search_Count_SeattleRent
Shift_Year,Shift_Month,
2003,10,1


In [139]:
merge_housing = merge_housing.drop(['RegionName','State','Metro', 'CountyName', 'SizeRank'], 1)

merge_housing  = pd.merge(merge_housing, google_trend_df_SeattleRent, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing.shape

(302, 8)

In [140]:

google_trend_df_SeattleHousing['Shift_Year'] = google_trend_df_SeattleHousing['Shift_Year'].astype(str)
google_trend_df_SeattleHousing['Shift_Month'] = google_trend_df_SeattleHousing['Shift_Month'].astype(str)

merge_housing  = pd.merge(merge_housing, google_trend_df_SeattleHousing, on=['Shift_Year', 'Shift_Month'], how='left')

google_trend_df_seattleApartment['Shift_Year'] = google_trend_df_seattleApartment['Shift_Year'].astype(str)
google_trend_df_seattleApartment['Shift_Month'] = google_trend_df_seattleApartment['Shift_Month'].astype(str)

merge_housing  = pd.merge(merge_housing, google_trend_df_seattleApartment, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment
0,Studio,2011,03,1097.5,2011,03,366200.0,45,32,25
1,Studio,2011,04,1095.0,2011,04,370800.0,48,27,22
2,Studio,2011,05,1025.0,2011,05,376000.0,46,32,24
3,Studio,2011,06,995.0,2011,06,382800.0,37,33,24
4,Studio,2011,07,1000.0,2011,07,373500.0,33,22,16


In [141]:
redfin_housing=pd.read_csv('data/House/Redfin_Seattle_All.csv')
redfin_housing['Region Type'].value_counts()
redfin_housing.head()

,City,Measure Display,Period Begin,Period End,Property Type,Region,Region Type,State,State Code,Table Id,...,New Listings Mom,New Listings Yoy,Number of Records,Period Duration,Price Drops,Price Drops Mom,Price Drops Yoy,Sold Above List,Sold Above List Mom,Sold Above List Yoy
0,Seattle,NaN,1/1/2012,1/31/2012,All Residential,"Seattle, WA",place,Washington,WA,16163,...,67.50%,-16.20%,1,30,0.286394,0.082724,-0.006008,0.162996,-0.008754,0.059369
1,Seattle,NaN,5/1/2015,5/31/2015,All Residential,"Seattle, WA",place,Washington,WA,16163,...,-0.90%,-1.60%,1,30,0.197441,-0.020950,-0.079781,0.561572,-0.002885,0.121801
2,Seattle,NaN,7/1/2017,7/31/2017,All Residential,"Seattle, WA",place,Washington,WA,16163,...,-12.00%,-8.10%,1,30,0.245077,-0.009731,-0.005338,0.595837,-0.055433,0.085781
3,Seattle,NaN,11/1/2014,11/30/2014,All Residential,"Seattle, WA",place,Washington,WA,16163,...,-39.90%,-1.20%,1,30,0.276562,-0.078092,-0.018133,0.277987,-0.051151,-0.004197
4,Seattle,NaN,7/1/2016,7/31/2016,All Residential,"Seattle, WA",place,Washington,WA,16163,...,-3.20%,11.40%,1,30,0.250415,-0.017460,0.019002,0.510056,-0.026529,0.023713


In [142]:
redfin_housing = redfin_housing.drop(['City','Measure Display','Period End','State','State Code', 'Table Id', 'Property Type','Period Duration', 'Region Type', 'Region', 'Worksheet Filter'],1)
redfin_housing.head()

,Period Begin,Avg Sale To List,Avg Sale To List Mom,Avg Sale To List Yoy,Homes Sold,Homes Sold Mom,Homes Sold Yoy,Inventory,Inventory Mom,Inventory Yoy,...,New Listings,New Listings Mom,New Listings Yoy,Number of Records,Price Drops,Price Drops Mom,Price Drops Yoy,Sold Above List,Sold Above List Mom,Sold Above List Yoy
0,1/1/2012,0.969093,-0.20%,0.90%,454,-27.10%,17.60%,"2,102",-3.60%,-28.70%,...,655,67.50%,-16.20%,1,0.286394,0.082724,-0.006008,0.162996,-0.008754,0.059369
1,5/1/2015,1.036981,0.30%,2.30%,"1,145",5.40%,9.50%,"1,094",4.80%,-26.90%,...,"1,394",-0.90%,-1.60%,1,0.197441,-0.020950,-0.079781,0.561572,-0.002885,0.121801
2,7/1/2017,1.055992,-1.70%,2.20%,"1,158",-10.90%,-6.90%,914,9.90%,-24.20%,...,"1,233",-12.00%,-8.10%,1,0.245077,-0.009731,-0.005338,0.595837,-0.055433,0.085781
3,11/1/2014,0.996997,-0.30%,0.30%,796,-22.90%,3.50%,"1,280",-18.90%,-16.20%,...,593,-39.90%,-1.20%,1,0.276562,-0.078092,-0.018133,0.277987,-0.051151,-0.004197
4,7/1/2016,1.034185,-1.00%,0.10%,"1,244",-13.30%,-5.60%,"1,206",15.00%,12.10%,...,"1,342",-3.20%,11.40%,1,0.250415,-0.017460,0.019002,0.510056,-0.026529,0.023713


In [143]:
redfin_housing ['Date'] = pd.to_datetime((redfin_housing['Period Begin']) ,format='%m/%d/%Y')
redfin_housing_shif = redfin_housing.Date+   pd.offsets.DateOffset(months=shift_month_d)
redfin_housing ['Shift_Year'] = redfin_housing_shif.dt.year.astype(str)
redfin_housing ['Shift_Month'] = redfin_housing_shif.dt.month.map("{:02}".format)
redfin_housing = redfin_housing.drop(['Period Begin', 'Date'], 1)


redfin_housing.head()

,Avg Sale To List,Avg Sale To List Mom,Avg Sale To List Yoy,Homes Sold,Homes Sold Mom,Homes Sold Yoy,Inventory,Inventory Mom,Inventory Yoy,Median Dom,...,New Listings Yoy,Number of Records,Price Drops,Price Drops Mom,Price Drops Yoy,Sold Above List,Sold Above List Mom,Sold Above List Yoy,Shift_Year,Shift_Month
0,0.969093,-0.20%,0.90%,454,-27.10%,17.60%,"2,102",-3.60%,-28.70%,59,...,-16.20%,1,0.286394,0.082724,-0.006008,0.162996,-0.008754,0.059369,2011,10
1,1.036981,0.30%,2.30%,"1,145",5.40%,9.50%,"1,094",4.80%,-26.90%,7,...,-1.60%,1,0.197441,-0.020950,-0.079781,0.561572,-0.002885,0.121801,2015,02
2,1.055992,-1.70%,2.20%,"1,158",-10.90%,-6.90%,914,9.90%,-24.20%,8,...,-8.10%,1,0.245077,-0.009731,-0.005338,0.595837,-0.055433,0.085781,2017,04
3,0.996997,-0.30%,0.30%,796,-22.90%,3.50%,"1,280",-18.90%,-16.20%,20,...,-1.20%,1,0.276562,-0.078092,-0.018133,0.277987,-0.051151,-0.004197,2014,08
4,1.034185,-1.00%,0.10%,"1,244",-13.30%,-5.60%,"1,206",15.00%,12.10%,8,...,11.40%,1,0.250415,-0.017460,0.019002,0.510056,-0.026529,0.023713,2016,04


In [144]:
merge_housing  = pd.merge(merge_housing, redfin_housing, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,New Listings,New Listings Mom,New Listings Yoy,Number of Records,Price Drops,Price Drops Mom,Price Drops Yoy,Sold Above List,Sold Above List Mom,Sold Above List Yoy
0,Studio,2011,03,1097.5,2011,03,366200.0,45,32,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Studio,2011,04,1095.0,2011,04,370800.0,48,27,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Studio,2011,05,1025.0,2011,05,376000.0,46,32,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Studio,2011,06,995.0,2011,06,382800.0,37,33,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Studio,2011,07,1000.0,2011,07,373500.0,33,22,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
merge_housing ['Date'] = pd.to_datetime((merge_housing['Year'] + '-' + merge_housing['Month']+ '-01') ,format='%Y-%m-%d')
merge_housing ['Date'].head()

0   2011-03-01
1   2011-04-01
2   2011-05-01
3   2011-06-01
4   2011-07-01
Name: Date, dtype: datetime64[ns]

In [146]:
merge_housing.shape

(302, 48)

# Google Crime Rate

In [147]:
with open("google_crime_news.pkl", 'rb') as picklefile: 
    google_crime_data = pickle.load(picklefile)
google_crime_data = pd.DataFrame(google_crime_data)
google_crime_data.head()

,count,month,term,year
0,36900,1,news:+seattle+shooting,2018
1,23200,2,news:+seattle+shooting,2018
2,31200,3,news:+seattle+shooting,2018
3,40200,4,news:+seattle+shooting,2018
4,8050,1,news:+seattle+shooting,2017


In [148]:
google_crime_data ['Date'] = pd.to_datetime((google_crime_data['year'].astype(str) + '-' + google_crime_data['month'].astype(str)+ '-01') ,format='%Y-%m-%d')
google_crime_data.head()

,count,month,term,year,Date
0,36900,1,news:+seattle+shooting,2018,2018-01-01
1,23200,2,news:+seattle+shooting,2018,2018-02-01
2,31200,3,news:+seattle+shooting,2018,2018-03-01
3,40200,4,news:+seattle+shooting,2018,2018-04-01
4,8050,1,news:+seattle+shooting,2017,2017-01-01


In [149]:
google_crime_data_shift = google_crime_data.Date+   pd.offsets.DateOffset(months=shift_month_d)
google_crime_data ['Shift_Year'] = google_crime_data_shift.dt.year
google_crime_data ['Shift_Month'] = google_crime_data_shift.dt.month.map("{:02}".format)
google_crime_data = google_crime_data.drop(['month', 'year', 'Date'], 1)


In [150]:
google_crime_data.head()

,count,term,Shift_Year,Shift_Month
0,36900,news:+seattle+shooting,2017,10
1,23200,news:+seattle+shooting,2017,11
2,31200,news:+seattle+shooting,2017,12
3,40200,news:+seattle+shooting,2018,01
4,8050,news:+seattle+shooting,2016,10


In [151]:
google_crime_data['term'].value_counts()

news:+seattle+murder           52
news:+seattle+dead+shooting    52
news:+seattle+shooting         52
Name: term, dtype: int64

In [152]:
def convertTerm(term, sortTerm):
    temp_df_new = google_crime_data[google_crime_data['term'] == term]
    temp_df_new = temp_df_new.rename(columns={'count': 'count_news_'+sortTerm})
    temp_df_new = temp_df_new.drop(['term'], 1)    
    temp_df_new['Shift_Year'] = temp_df_new['Shift_Year'].astype(str)
    temp_df_new['Shift_Month'] = temp_df_new['Shift_Month'].astype(str)

    return temp_df_new
google_news_shooting = convertTerm('news:+seattle+shooting', 'seattle+shooting')
google_news_murder = convertTerm('news:+seattle+murder', 'seattle+murder ')
google_news_dead = convertTerm('news:+seattle+dead+shooting', 'seattle+dead+shooting')
google_news_dead.shape

(52, 3)

In [153]:
merge_housing  = pd.merge(merge_housing, google_news_shooting, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing  = pd.merge(merge_housing, google_news_murder, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing  = pd.merge(merge_housing, google_news_dead, on=['Shift_Year', 'Shift_Month'], how='left')
t = merge_housing.dropna()
merge_housing.shape

(302, 51)

# import Stock data

In [154]:
amazon_stock=pd.read_csv('data/stock/AMZN.csv')
amazon_stock['Date'] = pd.to_datetime((amazon_stock['Date']) ,format='%Y-%m-%d')
amazon_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-01,175.889999,196.500000,173.500000,194.440002,194.440002,110536900
1,2012-02-01,173.809998,193.570007,172.000000,179.690002,179.690002,146580400
2,2012-03-01,179.889999,209.850006,176.580002,202.509995,202.509995,112975100
3,2012-04-01,198.020004,233.839996,183.649994,231.899994,231.899994,110535400
4,2012-05-01,229.399994,232.970001,207.110001,212.910004,212.910004,92774300


In [155]:
amazon_stock_shift = amazon_stock.Date+   pd.offsets.DateOffset(months=shift_month_d)
amazon_stock ['Shift_Year'] = amazon_stock_shift.dt.year
amazon_stock ['Shift_Month'] = amazon_stock_shift.dt.month.map("{:02}".format)
amazon_stock = amazon_stock.drop(['Date'], 1)
amazon_stock.head()


,Open,High,Low,Close,Adj Close,Volume,Shift_Year,Shift_Month
0,175.889999,196.500000,173.500000,194.440002,194.440002,110536900,2011,10
1,173.809998,193.570007,172.000000,179.690002,179.690002,146580400,2011,11
2,179.889999,209.850006,176.580002,202.509995,202.509995,112975100,2011,12
3,198.020004,233.839996,183.649994,231.899994,231.899994,110535400,2012,01
4,229.399994,232.970001,207.110001,212.910004,212.910004,92774300,2012,02


In [156]:
amazon_stock['Shift_Year'] = amazon_stock['Shift_Year'].astype(str)
amazon_stock['Shift_Month'] = amazon_stock['Shift_Month'].astype(str)
merge_housing  = pd.merge(merge_housing, amazon_stock, on=['Shift_Year', 'Shift_Month'], how='left')
merge_housing

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,Date,count_news_seattle+shooting,count_news_seattle+murder,count_news_seattle+dead+shooting,Open,High,Low,Close,Adj Close,Volume
0,Studio,2011,03,1097.5,2011,03,366200.0,45,32,25,...,2011-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Studio,2011,04,1095.0,2011,04,370800.0,48,27,22,...,2011-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Studio,2011,05,1025.0,2011,05,376000.0,46,32,24,...,2011-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Studio,2011,06,995.0,2011,06,382800.0,37,33,24,...,2011-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Studio,2011,07,1000.0,2011,07,373500.0,33,22,16,...,2011-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Studio,2011,08,1000.0,2011,08,368400.0,33,25,21,...,2011-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Studio,2011,09,1075.0,2011,09,359300.0,27,23,19,...,2011-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Studio,2011,10,995.0,2011,10,365300.0,36,25,22,...,2011-10-01,NaN,NaN,NaN,175.889999,196.500000,173.500000,194.440002,194.440002,110536900.0
8,Studio,2011,11,865.0,2011,11,367900.0,36,24,22,...,2011-11-01,NaN,NaN,NaN,173.809998,193.570007,172.000000,179.690002,179.690002,146580400.0
9,Studio,2011,12,902.0,2011,12,381900.0,38,28,22,...,2011-12-01,NaN,NaN,NaN,179.889999,209.850006,176.580002,202.509995,202.509995,112975100.0


In [157]:
with open('all_data.pkl', 'wb') as picklefile:
    pickle.dump(merge_housing, picklefile)

In [158]:
with open("all_data.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)
my_old_data.shape

(302, 57)

In [159]:
my_old_data.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,Date,count_news_seattle+shooting,count_news_seattle+murder,count_news_seattle+dead+shooting,Open,High,Low,Close,Adj Close,Volume
0,Studio,2011,03,1097.5,2011,03,366200.0,45,32,25,...,2011-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Studio,2011,04,1095.0,2011,04,370800.0,48,27,22,...,2011-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Studio,2011,05,1025.0,2011,05,376000.0,46,32,24,...,2011-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Studio,2011,06,995.0,2011,06,382800.0,37,33,24,...,2011-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Studio,2011,07,1000.0,2011,07,373500.0,33,22,16,...,2011-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
